In [1]:
pip install webdriver-manager


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: C:\Users\carlo\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd

# Configuración Selenium
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

url = "https://www.turismo.gal/localizador-de-recursos?langId=es_ES"
driver.get(url)

print("✅ Selecciona manualmente los filtros que quieras (municipio, interés, etc.)")
print("⏳ Tienes 30 segundos para hacerlo antes de que empiece la extracción...")

time.sleep(30)

wait = WebDriverWait(driver, 20)
data = []
pagina = 1

while True:
    print(f"📄 Extrayendo página {pagina}...")

    # Esperar resultados
    resultados = wait.until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.clearfix.list-item"))
    )

    for r in resultados:
        try:
            nombre_elem = r.find_element(By.CSS_SELECTOR, "a.title.mod-href")
            nombre = nombre_elem.text.strip()
            link = nombre_elem.get_attribute("href")  # enlace del nombre
        except:
            nombre = None
            link = None

        try:
            direccion = r.find_element(By.CSS_SELECTOR, "div.col-sm-12.address .media-body").text.strip()
        except:
            direccion = None

        try:
            detalle_raw = r.find_element(By.CSS_SELECTOR, "small.show").text.strip()
            if "-" in detalle_raw:
                partes = detalle_raw.split("-", 1)
                tipo = partes[0].strip()
                categoria = partes[1].strip()
            else:
                tipo = detalle_raw.strip()
                categoria = None
        except:
            detalle_raw = None
            tipo = None
            categoria = None

        try:
            telefono = r.find_element(By.CSS_SELECTOR, "div.telefono").text.strip()
        except:
            telefono = None

        try:
            email_elem = r.find_element(By.CSS_SELECTOR, "a[href^='mailto:']")
            email = email_elem.text.strip()
        except:
            email = None

        data.append({
            "nombre": nombre,
            "link": link,
            "direccion": direccion,
            "tipo": tipo,
            "categoria": categoria,
            "telefono": telefono,
            "email": email
        })

    # Verificar si el botón siguiente está deshabilitado
    try:
        next_button = driver.find_element(By.CSS_SELECTOR, "a.btn.btn-default.btn-pagination[title='Siguiente']")
        if "disabled" in next_button.get_attribute("class"):
            print("🚩 Última página alcanzada.")
            break
        else:
            driver.execute_script("arguments[0].click();", next_button)
            pagina += 1
            time.sleep(3)
    except:
        print("🚩 No se encontró botón siguiente. Fin.")
        break

# Guardar CSV
df = pd.DataFrame(data)
df.to_csv("ServiciosTurisiticos.csv", index=False, encoding="utf-8-sig")

print("✅ Datos guardados en ServiciosTurisiticos.csv")
driver.quit()


✅ Selecciona manualmente los filtros que quieras (municipio, interés, etc.)
⏳ Tienes 30 segundos para hacerlo antes de que empiece la extracción...
📄 Extrayendo página 1...
📄 Extrayendo página 2...
📄 Extrayendo página 3...
📄 Extrayendo página 4...
📄 Extrayendo página 5...
📄 Extrayendo página 6...
📄 Extrayendo página 7...
📄 Extrayendo página 8...
📄 Extrayendo página 9...
📄 Extrayendo página 10...
📄 Extrayendo página 11...
📄 Extrayendo página 12...
📄 Extrayendo página 13...
📄 Extrayendo página 14...
📄 Extrayendo página 15...
📄 Extrayendo página 16...
📄 Extrayendo página 17...
📄 Extrayendo página 18...
📄 Extrayendo página 19...
📄 Extrayendo página 20...
📄 Extrayendo página 21...
📄 Extrayendo página 22...
📄 Extrayendo página 23...
📄 Extrayendo página 24...
📄 Extrayendo página 25...
📄 Extrayendo página 26...
🚩 Última página alcanzada.
✅ Datos guardados en ServiciosTurisiticos.csv
